# Ranking Objects on Blocket Bostad 
### Variables
- Address
- Area
- Type
- Rooms
- Size in square meters
- Move in date
- Move out date
- Rent in SEK per month
- Address + area in one string
- Rent per square meters
- Distance to Stockholm City in km
- Travel time to Stockholm City by public transport

### Three ranking variables in the dataframe:
- rent/sqm_rank:
    - Creating 10 quantiles out of the rent/sqm, where the lowest == 1 and highest == 10
- move_in_difference_rank
    - Number of days between the move in date and the desired move in date specified by the user, divided by ten and rounded to the closest integer
- distance_time_rank
    - Creating 10 quantiles out of the travel time to city centre with public transport, where the lowest == 1 and highest == 10
    
The sum of these three ranking variables are stored in `df['rank_sum']`, and the rental object with the lowest rank_sum is considered the "best" object based on these attributes.

# Code
Separate cells for different parts of the code

## Libraries

In [41]:
import pandas as pd
import requests
import time 
from bs4 import BeautifulSoup as bs

In [134]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

In [404]:
import itertools

In [604]:
import numpy as np

## Google Maps API

In [ ]:
pip install -U googlemaps

In [ ]:
import googlemaps
from datetime import datetime

In [ ]:
# Google Maps API object
api_key = '...try next time'
gmaps = googlemaps.Client(key = api_key)

## Web Scraping

In [2]:
# Prompt user to enter different filters
move_in_user_date = input("When do you want to move in to the rental object? (Format: 2022-01-01): ")
area_input = input("In what area are you looking for a rental: ")
accomodation_type = input("Are you looking for a private or a shared rental, or both? (type p for private, s for shared, b for both): ")
furnished = input("Are you looking for a furnished or an unfurnished rental, or both? (f for furnished, u for unfurnished, b for both): ")
rooms_input = input("What is the minimal amount of rooms you are looking for?: ")
max_rent = input("What is your max rent per month?: ")

# Create empty lists to fill with values from scraping
address_list = []
area_list = []
obj_info = []
type_list = []
rooms_list = []
size_list = []
moveinfo_list = []
movein_list = []
moveout_list = []
price_list = []

# Retrieve url with the function
url = generate_url(area_input ,furnished, accomodation_type, rooms_input, max_rent)

# Create the driver
driver = webdriver.Chrome("/Applications/chromedriver")
driver.maximize_window()
driver.get(url)

# Get around cookies
element =  driver.find_element(By.XPATH, '//*[@id="radix-0"]/footer/div/button')
WebDriverWait(driver, 2) # make web wait 2 sek
element.click()
WebDriverWait(driver, 2) # make web wait 2 sek

# Get number of pages
soup = bs(driver.page_source, "lxml")
WebDriverWait(driver, 10).until(lambda x: x.find_element(By.CSS_SELECTOR, "div.stack__Wrapper-sc-11qezct-0.jWpksc"))
page_list = soup.select('div.stack__Wrapper-sc-11qezct-0.jWpksc')

if page_list[0].text.count(".") == 3 and page_list[0].text[-2] != ".": # Checks if max page number is 1 or 2 digits
    page_number = page_list[0].text[-2:]
else:
    page_number = page_list[0].text[-1]

page_number_max = int(page_number)


# Loops over the number of pages
for page in range(1, page_number_max + 1):
    # Waits until the page is loaded
    WebDriverWait(driver, 10).until(lambda x: x.find_element(By.CSS_SELECTOR, "h2.typography__Element-sc-n5ojo8-0.krdzXK"))
    
    # Creates soup
    soup = bs(driver.page_source, "lxml") 
    
    # Scrapes values and appends to lists
    address_list.append([i.text for i in soup.select("h2.typography__Element-sc-n5ojo8-0.krdzXK")])
    area_list.append([i.text for i in soup.select("p.typography__Element-sc-n5ojo8-0.htKkRQ.home-item__TypographyElipsed-sc-ygluq4-6.home-item___StyledTypographyElipsed-sc-ygluq4-7.djQoTe.iomSqL")])
    obj_info.append([i.text for i in soup.select("li.inline-bullet-list__InlineBulletListItem-sc-d74tjp-1.ftQLRx")])
    moveinfo_list.append([i.text for i in soup.select("p.typography__Element-sc-n5ojo8-0.cULIwk.home-item__Duration-sc-ygluq4-1.kgrCpi")])
    price_list.append([i.text.replace("\xa0","")[:-2] for i in soup.select("span.typography__Element-sc-n5ojo8-0.krdzXK.home-item__Rent-sc-ygluq4-0.iTgUyM")])  
    
    # Clicks the next page button
    next_button = driver.find_element(By.CSS_SELECTOR,'[aria-label = Nästa]')
    next_button.click()




KeyboardInterrupt: Interrupted by user

## Create DataFrame

### Manipulate data

In [869]:
# Split obj info:
for e in obj_info:
    type_list.append(e[::3])
    rooms_list.append(e[1::3])
    size_list.append(e[2::3])

# Split moveinfo:
for e in moveinfo_list:
    for row in e:
        if row[0] == "S":
            movein_list.append(row[:15])
            moveout_list.append(row[15:])
        else:
            movein_list.append(row[:10])
            moveout_list.append(row[10:])            

# Fix rest of variablies:
address_list = list(itertools.chain.from_iterable(address_list))
area_list = list(itertools.chain.from_iterable(area_list))
type_list = list(itertools.chain.from_iterable(type_list))
rooms_list = list(itertools.chain.from_iterable(rooms_list))
size_list = list(itertools.chain.from_iterable(size_list))
price_list = list(itertools.chain.from_iterable(price_list))

### Check Length of Columns

In [870]:
print(len(address_list))
print(len(area_list))
print(len(type_list))
print(len(rooms_list))
print(len(size_list))
print(len(movein_list))
print(len(moveout_list))
print(len(price_list))

208
208
208
208
208
208
208
208


### Create DataFrame

#### Functions

In [1]:
def fix_string_to_int(string):
    """Takes a string and returns the first number of connected integers in the string as an int"""
    temp_string = []
    for index,char in enumerate(string):
        if char.isdigit() == False:
            temp_string.append(string[:index])
            break      
        else:
            pass
        
    temp_int = "".join(temp_string)
    return int(temp_int)

def fix_string_to_float(string):
    """Takes a string and returns the first number of "."-connected integers in the string as a float"""
    temp_string = []
    for index,char in enumerate(string):
        if char == " ":
            temp_string.append(string[:index])
            break      
        else:
            pass
        
    temp_float = "".join(temp_string)
    return float(temp_float)


def get_distance_in_transit(start, stop):
    """Takes two strings and Google Directions API and returns travel 
    time in public transportation between start and stop"""
    now = datetime.now()
    try:
        directions_result = gmaps.directions(start,
                                             stop,
                                             mode="transit",
                                             departure_time=now)
        return directions_result[0]["legs"][0]["duration"]["text"]
    except:
        return np.nan
    
def get_distance_in_km(start, stop):
    """Takes two strings and returns the distance between start and stop in km, 
    using Google Directions API"""
    now = datetime.now()
    try:
        directions_result = gmaps.directions(start,
                                             stop,
                                             mode="transit",
                                             departure_time=now)
        return directions_result[0]["legs"][0]["distance"]["text"]
    except:
        return np.nan
    
def generate_url(area, furnished, acco_type, rooms, max_rent):
    """Takes 5 different filter arguments for Blocket Bostad and returns the correct 
    URL-string given these arguments"""
    if furnished == "f":
        if acco_type == "p":
            return f"https://bostad.blocket.se/p2/sv/find-home/?furnished=furnished&maxMonthlyCost={max_rent}&rooms={rooms}&searchAreas={area}&sharedHome=privateHome"
        elif acco_type == "s":
            return f"https://bostad.blocket.se/p2/sv/find-home/?furnished=furnished&maxMonthlyCost={max_rent}&rooms={rooms}&searchAreas={area}&sharedHome=sharedHome"
        else:
            return f"https://bostad.blocket.se/p2/sv/find-home/?furnished=furnished&maxMonthlyCost={max_rent}&rooms={rooms}&searchAreas={area}&sharedHome=privateHome,sharedHome"
    elif furnished == "u":
        if acco_type == "p":
            return f"https://bostad.blocket.se/p2/sv/find-home/?furnished=unfurnished&maxMonthlyCost={max_rent}rooms={rooms}&searchAreas={area}&sharedHome=privateHome"
        elif acco_type == "s":
            return f"https://bostad.blocket.se/p2/sv/find-home/?furnished=unfurnished&maxMonthlyCost={max_rent}&rooms={rooms}&searchAreas={area}&sharedHome=sharedHome"
        else:
            return f"https://bostad.blocket.se/p2/sv/find-home/?furnished=unfurnished&maxMonthlyCost={max_rent}&rooms={rooms}&searchAreas={area}&sharedHome=privateHome,sharedHome"
    else:
        if acco_type == "p":
            return f"https://bostad.blocket.se/p2/sv/find-home/?furnished=furnished,unfurnished&maxMonthlyCost={max_rent}&rooms={rooms}&searchAreas={area}&sharedHome=privateHome"
        elif acco_type == "s":
            return f"https://bostad.blocket.se/p2/sv/find-home/?furnished=furnished,unfurnished&maxMonthlyCost={max_rent}&rooms={rooms}&searchAreas={area}&sharedHome=sharedHome"
        else:
            return f"https://bostad.blocket.se/p2/sv/find-home/?furnished=furnished,unfurnished&maxMonthlyCost={max_rent}&rooms={rooms}&searchAreas={area}&sharedHome=privateHome,sharedHome"
    
    
    
    
    

#### DataFrame

In [872]:
# Create dictionary
df_dict = {"address": address_list, "area": area_list, "type": type_list, "rooms": rooms_list,
      "size_sqm": size_list, "move_in": movein_list, "move_out": moveout_list, "rent": price_list}

# Create df from dict
df = pd.DataFrame(df_dict)

# Manipulate df values
df['size_sqm'] = df['size_sqm'].apply(fix_string_to_int)
df['rooms'] = df['rooms'].apply(fix_string_to_int)
df['rent'] = pd.to_numeric(df['rent'])
df["move_in"] = df["move_in"].apply(lambda x: "2022-10-31" if x == "Snarast möjligt" else x)
df["move_in"] = pd.to_datetime(df["move_in"])
df["address_area"] = df["address"] + ", " +  df["area"]
df["move_in_difference"] = abs(df["move_in"] - datetime.strptime(move_in_user_date, '%Y-%m-%d'))
df["move_in_difference"] = df["move_in_difference"].astype(str)
df['rent_per_sqm(kr)'] = df["rent"] / df["size_sqm"]
df["distance_time_transit"] = df["address_area"].apply(get_distance_in_transit, args = (area_input,))
df["distance_km"] = df["address_area"].apply(get_distance_in_km, args = (area_input,))
df["rent/sqm_rank"] = pd.qcut(df["rent_per_sqm(kr)"], 10, [1,2,3,4,5,6,7,8,9,10])
df["rent/sqm_rank"] = df["rent/sqm_rank"].astype(int)

# Drop all na, generally only exists when google api cannot find the adress
df = df.dropna()

# Fix move in date rank:
df["move_in_difference_rank"] = df['move_in_difference'].apply(fix_string_to_int)
df["move_in_difference_rank"] = round(df["move_in_difference_rank"] / 10)
df["move_in_difference_rank"] = df["move_in_difference_rank"].astype(int)

# Fix distance rank:
df["distance_km"] = df['distance_km'].apply(fix_string_to_int)
df["distance_km_rank"] = pd.qcut(df["distance_km"], 10, [1,2,3,4,5,6,7,8,9,10])
df["distance_km_rank"] = df["distance_km_rank"].astype(int)

# Fix travel time rank:
df["distance_time_rank"] = df['distance_time_transit'].apply(fix_string_to_float)
df["distance_time_rank"] = pd.qcut(df["distance_time_rank"], 10, [1,2,3,4,5,6,7,8,9,10])
df["distance_time_rank"] = df['distance_time_rank'].astype(int)

# Create rank sum
df["rank_sum"] = df["move_in_difference_rank"] + df["rent/sqm_rank"] + df["distance_km_rank"] + df["distance_time_rank"]

# Sort df
df.sort_values("rank_sum", inplace = True)

## Print DataFrame

In [873]:
df.head()

,address,area,type,rooms,size_sqm,move_in,move_out,rent,address_area,move_in_difference,rent_per_sqm(kr),distance_time_transit,distance_km,rent/sqm_rank,move_in_difference_rank,distance_km_rank,distance_time_rank,rank_sum
0,Hammarögatan,Farsta,Lägenhet,2,55,2022-11-01,Tillsvidare,12273,"Hammarögatan, Farsta",30 days,223.145455,28 mins,10,4,3,9,9,25
1,Ada Nilssons gata,Hägersten,Lägenhet,2,41,2022-10-31,Tillsvidare,10495,"Ada Nilssons gata, Hägersten",31 days,255.975610,26 mins,9,6,3,8,9,26
2,Höstgatan,Hägersten,Lägenhet,2,50,2023-01-16,2023-03-16,13383,"Höstgatan, Hägersten",46 days,267.660000,15 mins,5,6,5,4,3,18
3,Årstavägen,Årsta,Lägenhet,2,50,2022-10-31,2023-04-30,14693,"Årstavägen, Årsta",31 days,293.860000,20 mins,7,7,3,6,5,21
4,Dovregatan,Kista,Lägenhet,2,65,2022-11-07,Tillsvidare,10285,"Dovregatan, Kista",24 days,158.230769,12 mins,3,1,2,2,2,7


In [874]:
df.tail()

,address,area,type,rooms,size_sqm,move_in,move_out,rent,address_area,move_in_difference,rent_per_sqm(kr),distance_time_transit,distance_km,rent/sqm_rank,move_in_difference_rank,distance_km_rank,distance_time_rank,rank_sum
45,Främlingsvägen,Hägersten,Lägenhet,2,57,2023-01-02,2024-01-02,11544,"Främlingsvägen, Hägersten",32 days,202.526316,15 mins,5,4,3,4,3,14
46,Abrahamsbergsvägen,Stockholm,Lägenhet,2,45,2023-01-01,2023-09-03,7346,"Abrahamsbergsvägen , Stockholm",31 days,163.244444,24 mins,9,2,3,8,8,21
47,Stora Sällskapets Väg,Skärholmen,Lägenhet,2,65,2022-10-31,2023-06-15,9340,"Stora Sällskapets Väg, Skärholmen",31 days,143.692308,25 mins,9,1,3,8,8,20
48,Nockeby backe,Bromma,Lägenhet,2,51,2022-12-01,Tillsvidare,15000,"Nockeby backe, Bromma",0 days,294.117647,33 mins,13,7,0,10,10,27
49,Trollesundsvägen,Stockholm,Lägenhet,2,50,2022-10-31,Tillsvidare,13643,"Trollesundsvägen , Stockholm",31 days,272.860000,24 mins,8,7,3,7,8,25


# Run the Program
One cell with all of the code, running the entire program

In [905]:
move_in_user_date = input("When do you want to move in to the rental object? (Format: 2022-01-01): ")
area_input = input("In what area are you looking for a rental: ")
accomodation_type = input("Are you looking for a private or a shared rental, or both? (type p for private, s for shared, b for both): ")
furnished = input("Are you looking for a furnished or an unfurnished rental, or both? (f for furnished, u for unfurnished, b for both): ")
rooms_input = input("What is the minimal amount of rooms you are looking for?: ")
max_rent = input("What is your max rent per month?: ")

 
address_list = []
area_list = []
obj_info = []
type_list = []
rooms_list = []
size_list = []
moveinfo_list = []
movein_list = []
moveout_list = []
price_list = []
url = generate_url(area_input ,furnished, accomodation_type, rooms_input, max_rent)
driver = webdriver.Chrome("/Applications/chromedriver")
driver.maximize_window()
driver.get(url)

# Get around cookies
#WebDriverWait(driver, 2) # make web wait 2 sek
element =  driver.find_element(By.XPATH, '//*[@id="radix-0"]/footer/div/button')
WebDriverWait(driver, 2) # make web wait 2 sek
element.click()
WebDriverWait(driver, 2) # make web wait 2 sek

# SKIP PAGE NUMBER PROBLEM
"""# Get number of pages
soup = bs(driver.page_source, "lxml")
page_list = soup.select('div.stack__Wrapper-sc-11qezct-0.jWpksc > a:nth-child(7)')
WebDriverWait(driver, 10).until(lambda x: x.find_element(By.CSS_SELECTOR, "div.stack__Wrapper-sc-11qezct-0.jWpksc"))
page_list = soup.select('div.stack__Wrapper-sc-11qezct-0.jWpksc')
if page_list[0].text.count(".") == 3 and page_list[0].text[-2] != ".": 
    page_number = page_list[0].text[-2:]
else:
    page_number = page_list[0].text[-1]

page_number_max = int(page_number)"""

# Scrape only 5 pages, takes to long to load for a demo otherwise
for page in range(1, 6):
    WebDriverWait(driver, 10).until(lambda x: x.find_element(By.CSS_SELECTOR, "h2.typography__Element-sc-n5ojo8-0.krdzXK"))
    soup = bs(driver.page_source, "lxml")
    address_list.append([i.text for i in soup.select("h2.typography__Element-sc-n5ojo8-0.krdzXK")])
    area_list.append([i.text for i in soup.select("p.typography__Element-sc-n5ojo8-0.htKkRQ.home-item__TypographyElipsed-sc-ygluq4-6.home-item___StyledTypographyElipsed-sc-ygluq4-7.djQoTe.iomSqL")])
    obj_info.append([i.text for i in soup.select("li.inline-bullet-list__InlineBulletListItem-sc-d74tjp-1.ftQLRx")])
    moveinfo_list.append([i.text for i in soup.select("p.typography__Element-sc-n5ojo8-0.cULIwk.home-item__Duration-sc-ygluq4-1.kgrCpi")])
    price_list.append([i.text.replace("\xa0","")[:-2] for i in soup.select("span.typography__Element-sc-n5ojo8-0.krdzXK.home-item__Rent-sc-ygluq4-0.iTgUyM")])  
    next_button = driver.find_element(By.CSS_SELECTOR,'[aria-label = Nästa]')
    next_button.click()


# Split obj info:
for e in obj_info:
    type_list.append(e[::3])
    rooms_list.append(e[1::3])
    size_list.append(e[2::3])

# Split moveinfo:
for e in moveinfo_list:
    for row in e:
        if row[0] == "S":
            movein_list.append(row[:15])
            moveout_list.append(row[15:])
        else:
            movein_list.append(row[:10])
            moveout_list.append(row[10:])            

# Fix rest of variablies:
address_list = list(itertools.chain.from_iterable(address_list))
area_list = list(itertools.chain.from_iterable(area_list))
type_list = list(itertools.chain.from_iterable(type_list))
rooms_list = list(itertools.chain.from_iterable(rooms_list))
size_list = list(itertools.chain.from_iterable(size_list))
price_list = list(itertools.chain.from_iterable(price_list))

# Create dictionary
df_dict = {"address": address_list, "area": area_list, "type": type_list, "rooms": rooms_list,
      "size_sqm": size_list, "move_in": movein_list, "move_out": moveout_list, "rent": price_list}

# Create df from dict
df = pd.DataFrame(df_dict)

# Manipulate df values
df['size_sqm'] = df['size_sqm'].apply(fix_string_to_int)
df['rooms'] = df['rooms'].apply(fix_string_to_int)
df['rent'] = pd.to_numeric(df['rent'])
df["move_in"] = df["move_in"].apply(lambda x: "2022-10-31" if x == "Snarast möjligt" else x)
df["move_in"] = pd.to_datetime(df["move_in"])
df["address_area"] = df["address"] + ", " +  df["area"]
df["move_in_difference"] = abs(df["move_in"] - datetime.strptime(move_in_user_date, '%Y-%m-%d'))
df["move_in_difference"] = df["move_in_difference"].astype(str)
df['rent_per_sqm(kr)'] = df["rent"] / df["size_sqm"]
df["distance_time_transit"] = df["address_area"].apply(get_distance_in_transit, args = (area_input,))
df["distance_km"] = df["address_area"].apply(get_distance_in_km, args = (area_input,))
df["rent/sqm_rank"] = pd.qcut(df["rent_per_sqm(kr)"], 10, [1,2,3,4,5,6,7,8,9,10])
df["rent/sqm_rank"] = df["rent/sqm_rank"].astype(int)
df = df.dropna()

# Fix move in date rank:
df["move_in_difference_rank"] = df['move_in_difference'].apply(fix_string_to_int)
df["move_in_difference_rank"] = round(df["move_in_difference_rank"] / 10)
df["move_in_difference_rank"] = df["move_in_difference_rank"].astype(int)

# Fix travel time rank:
df["distance_time_rank"] = df['distance_time_transit'].apply(fix_string_to_float)
df["distance_time_rank"] = pd.qcut(df["distance_time_rank"], 10, [1,2,3,4,5,6,7,8,9,10])
df["distance_time_rank"] = df['distance_time_rank'].astype(int)

# Create rank sum
df["rank_sum"] = df["move_in_difference_rank"] + df["rent/sqm_rank"] + df["distance_time_rank"]

# Sort df
df.sort_values("rank_sum", inplace = True)
df.reset_index(drop = True, inplace = True)

When do you want to move in? (Format: 2022-01-01): 2022-12-01
Where are you looking for an apartment?: Stockholm
Do you want a private or shared apartment? (type p for private, s for shared, b for both): b
Do you want the residence to be furnished? (y for yes, n for n, b for both): b
Minimum amount of rooms?: 1
What is your max rent per month?: 20000


/var/folders/f2/gy3wvk2x3t36m50bbjptxdb80000gn/T/ipykernel_30447/28495966.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/Applications/chromedriver")


In [908]:
# Print
df.head(20)

,address,area,type,rooms,size_sqm,move_in,move_out,rent,address_area,move_in_difference,rent_per_sqm(kr),distance_time_transit,distance_km,rent/sqm_rank,move_in_difference_rank,distance_time_rank,rank_sum
0,Vattenledningsvägen,Hägersten,Lägenhet,2,50,2022-12-01,2023-03-31,11019,"Vattenledningsvägen, Hägersten",0 days,220.380000,12 mins,5.0 km,2,0,3,5
1,Dovregatan,Kista,Lägenhet,2,65,2022-11-07,Tillsvidare,10285,"Dovregatan, Kista",24 days,158.230769,11 mins,3.2 km,1,2,3,6
2,Hövdingagatan,Stockholm,Lägenhet,2,56,2022-12-06,2023-02-22,8081,"Hövdingagatan, Stockholm",5 days,144.303571,15 mins,6.0 km,1,0,5,6
3,Telefonplan,Hägersten,Lägenhet,1,40,2022-12-01,2023-05-31,14693,"Telefonplan, Hägersten",0 days,367.325000,10 mins,5.6 km,5,0,2,7
4,Wollmar Yxkullsgatan,Stockholm,Lägenhet,2,44,2022-12-01,2023-06-01,17841,"Wollmar Yxkullsgatan, Stockholm",0 days,405.477273,2 mins,1.3 km,6,0,1,7
5,Valhallavägen,Stockholm,Rum i Lägenhet,1,26,2022-12-01,Tillsvidare,9970,"Valhallavägen, Stockholm",0 days,383.461538,1 min,0.9 km,6,0,1,7
6,Finn Malmgrens väg,Johanneshov,Lägenhet,2,44,2022-12-15,Tillsvidare,14824,"Finn Malmgrens väg, Johanneshov",14 days,336.909091,10 mins,4.0 km,5,1,2,8
7,Bjurholmsgatan,Stockholm,Lägenhet,1,40,2022-11-12,Tillsvidare,13799,"Bjurholmsgatan, Stockholm",19 days,344.975000,8 mins,1.8 km,5,2,1,8
8,Larsbodavägen,Farsta,Rum i Lägenhet,1,35,2022-12-01,Tillsvidare,6297,"Larsbodavägen, Farsta",0 days,179.914286,20 mins,9.6 km,1,0,7,8
9,Hägerstensvägen,Hägersten,Lägenhet,2,41,2022-12-01,2023-11-30,15217,"Hägerstensvägen, Hägersten",0 days,371.146341,11 mins,5.6 km,5,0,3,8


In [907]:
df.tail()

,address,area,type,rooms,size_sqm,move_in,move_out,rent,address_area,move_in_difference,rent_per_sqm(kr),distance_time_transit,distance_km,rent/sqm_rank,move_in_difference_rank,distance_time_rank,rank_sum
93,Gävlegatan,Stockholm,Lägenhet,1,33,2022-10-31,Tillsvidare,15217,"Gävlegatan, Stockholm",31 days,461.121212,27 mins,5.9 km,8,3,9,20
94,Albanovägen,Stockholm,Lägenhet,1,22,2022-10-31,Tillsvidare,9970,"Albanovägen, Stockholm",31 days,453.181818,29 mins,5.2 km,8,3,10,21
95,Vårbergsvägen,Skärholmen,Rum i Villa,1,12,2022-10-31,Tillsvidare,5247,"Vårbergsvägen, Skärholmen",31 days,437.250000,30 mins,13.3 km,8,3,10,21
96,Vårbergsvägen,Skärholmen,Rum i Villa,1,10,2022-10-31,Tillsvidare,5037,"Vårbergsvägen, Skärholmen",31 days,503.700000,30 mins,13.3 km,9,3,10,22
97,backskåran,BROMMA,Rum i Lägenhet,1,11,2023-02-01,Tillsvidare,5352,"backskåran, BROMMA",62 days,486.545455,36 mins,11.9 km,9,6,10,25
